In [1]:
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob as glob
import os
import pandas as pd
from skimage import io, filters, morphology
import cv2
from scipy import ndimage as ndi
from pathlib import Path
%matplotlib inline
os.chdir('/Users/dylanschaff/Library/CloudStorage/GoogleDrive-dyschaff@sydshafferlab.com/My Drive/Schaff_Shared/Cloud/Schaff_paper/Data/DLS083_incucyte_data')
Image.MAX_IMAGE_PIXELS = 933120000

In [21]:
Path(os.getcwd()+'/percent_confluence').mkdir(parents=True, exist_ok=True)


In [2]:
file_list = glob.glob('/Users/dylanschaff/Library/CloudStorage/GoogleDrive-dyschaff@sydshafferlab.com/My Drive/Schaff_Shared/Cloud/Schaff_paper/Data/DLS083_incucyte_data/Images/*.tif')

In [3]:
len(file_list)


372

All conditions

In [9]:
# Get the tiffs to process
files = file_list[0:20]
#files = file_list[18:20] #for checking that the pipeline is working correctly 

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_0to20.csv')

DLS083_Plate1_Green_A2_1_11d00h00m
DLS083_Plate1_Green_A2_1_12d00h00m
DLS083_Plate1_Green_A2_1_13d00h00m
DLS083_Plate1_Green_A2_1_14d00h00m
DLS083_Plate1_Green_A2_1_15d00h00m
DLS083_Plate1_Green_A2_1_16d00h00m
DLS083_Plate1_Green_A2_1_17d00h00m
DLS083_Plate1_Green_A2_1_18d00h00m
DLS083_Plate1_Green_A2_1_19d00h00m
DLS083_Plate1_Green_A2_1_20d00h00m
DLS083_Plate1_Green_A2_1_21d00h00m
DLS083_Plate1_Green_A2_1_22d00h00m
DLS083_Plate1_Green_A2_1_23d00h00m
DLS083_Plate1_Green_A2_1_24d00h00m
DLS083_Plate1_Green_A2_1_25d00h00m
DLS083_Plate1_Green_A2_1_26d00h00m
DLS083_Plate1_Green_A2_1_27d00h00m
DLS083_Plate1_Green_A2_1_28d00h00m
DLS083_Plate1_Green_A2_1_29d00h00m
DLS083_Plate1_Green_A2_1_30d00h00m


In [10]:
# Get the tiffs to process
files = file_list[20:40]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_20to40.csv')

DLS083_Plate1_Green_A3_1_00d00h00m
DLS083_Plate1_Green_A3_1_01d00h00m
DLS083_Plate1_Green_A3_1_02d00h00m
DLS083_Plate1_Green_A3_1_03d00h00m
DLS083_Plate1_Green_A3_1_04d00h00m
DLS083_Plate1_Green_A3_1_05d00h00m
DLS083_Plate1_Green_A3_1_06d00h00m
DLS083_Plate1_Green_A3_1_07d00h00m
DLS083_Plate1_Green_A3_1_08d00h00m
DLS083_Plate1_Green_A3_1_09d00h00m
DLS083_Plate1_Green_A3_1_10d00h00m
DLS083_Plate1_Green_A3_1_11d00h00m
DLS083_Plate1_Green_A3_1_12d00h00m
DLS083_Plate1_Green_A3_1_13d00h00m
DLS083_Plate1_Green_A3_1_14d00h00m
DLS083_Plate1_Green_A3_1_15d00h00m
DLS083_Plate1_Green_A3_1_16d00h00m
DLS083_Plate1_Green_A3_1_17d00h00m
DLS083_Plate1_Green_A3_1_18d00h00m
DLS083_Plate1_Green_A3_1_19d00h00m


In [11]:
# Get the tiffs to process
files = file_list[40:60]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_40to60.csv')

DLS083_Plate1_Green_A3_1_20d00h00m
DLS083_Plate1_Green_A3_1_21d00h00m
DLS083_Plate1_Green_A3_1_22d00h00m
DLS083_Plate1_Green_A3_1_23d00h00m
DLS083_Plate1_Green_A3_1_24d00h00m
DLS083_Plate1_Green_A3_1_25d00h00m
DLS083_Plate1_Green_A3_1_26d00h00m
DLS083_Plate1_Green_A3_1_27d00h00m
DLS083_Plate1_Green_A3_1_28d00h00m
DLS083_Plate1_Green_A3_1_29d00h00m
DLS083_Plate1_Green_A3_1_30d00h00m
DLS083_Plate1_Green_B1_1_00d00h00m
DLS083_Plate1_Green_B1_1_01d00h00m
DLS083_Plate1_Green_B1_1_02d00h00m
DLS083_Plate1_Green_B1_1_03d00h00m
DLS083_Plate1_Green_B1_1_04d00h00m
DLS083_Plate1_Green_B1_1_05d00h00m
DLS083_Plate1_Green_B1_1_06d00h00m
DLS083_Plate1_Green_B1_1_07d00h00m
DLS083_Plate1_Green_B1_1_08d00h00m


In [12]:
# Get the tiffs to process
files = file_list[60:80]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_60to80.csv')

DLS083_Plate1_Green_B1_1_09d00h00m
DLS083_Plate1_Green_B1_1_10d00h00m
DLS083_Plate1_Green_B1_1_11d00h00m
DLS083_Plate1_Green_B1_1_12d00h00m
DLS083_Plate1_Green_B1_1_13d00h00m
DLS083_Plate1_Green_B1_1_14d00h00m
DLS083_Plate1_Green_B1_1_15d00h00m
DLS083_Plate1_Green_B1_1_16d00h00m
DLS083_Plate1_Green_B1_1_17d00h00m
DLS083_Plate1_Green_B1_1_18d00h00m
DLS083_Plate1_Green_B1_1_19d00h00m
DLS083_Plate1_Green_B1_1_20d00h00m
DLS083_Plate1_Green_B1_1_21d00h00m
DLS083_Plate1_Green_B1_1_22d00h00m
DLS083_Plate1_Green_B1_1_23d00h00m
DLS083_Plate1_Green_B1_1_24d00h00m
DLS083_Plate1_Green_B1_1_25d00h00m
DLS083_Plate1_Green_B1_1_26d00h00m
DLS083_Plate1_Green_B1_1_27d00h00m
DLS083_Plate1_Green_B1_1_28d00h00m


In [13]:
# Get the tiffs to process
files = file_list[80:100]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_80to100.csv')

DLS083_Plate1_Green_B1_1_29d00h00m
DLS083_Plate1_Green_B1_1_30d00h00m
DLS083_Plate1_Green_B2_1_00d00h00m
DLS083_Plate1_Green_B2_1_01d00h00m
DLS083_Plate1_Green_B2_1_02d00h00m
DLS083_Plate1_Green_B2_1_03d00h00m
DLS083_Plate1_Green_B2_1_04d00h00m
DLS083_Plate1_Green_B2_1_05d00h00m
DLS083_Plate1_Green_B2_1_06d00h00m
DLS083_Plate1_Green_B2_1_07d00h00m
DLS083_Plate1_Green_B2_1_08d00h00m
DLS083_Plate1_Green_B2_1_09d00h00m
DLS083_Plate1_Green_B2_1_10d00h00m
DLS083_Plate1_Green_B2_1_11d00h00m
DLS083_Plate1_Green_B2_1_12d00h00m
DLS083_Plate1_Green_B2_1_13d00h00m
DLS083_Plate1_Green_B2_1_14d00h00m
DLS083_Plate1_Green_B2_1_15d00h00m
DLS083_Plate1_Green_B2_1_16d00h00m
DLS083_Plate1_Green_B2_1_17d00h00m


In [14]:
# Get the tiffs to process
files = file_list[100:120]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_100to120.csv')

DLS083_Plate1_Green_B2_1_18d00h00m
DLS083_Plate1_Green_B2_1_19d00h00m
DLS083_Plate1_Green_B2_1_20d00h00m
DLS083_Plate1_Green_B2_1_21d00h00m
DLS083_Plate1_Green_B2_1_22d00h00m
DLS083_Plate1_Green_B2_1_23d00h00m
DLS083_Plate1_Green_B2_1_24d00h00m
DLS083_Plate1_Green_B2_1_25d00h00m
DLS083_Plate1_Green_B2_1_26d00h00m
DLS083_Plate1_Green_B2_1_27d00h00m
DLS083_Plate1_Green_B2_1_28d00h00m
DLS083_Plate1_Green_B2_1_29d00h00m
DLS083_Plate1_Green_B2_1_30d00h00m
DLS083_Plate1_Green_B3_1_00d00h00m
DLS083_Plate1_Green_B3_1_01d00h00m
DLS083_Plate1_Green_B3_1_02d00h00m
DLS083_Plate1_Green_B3_1_03d00h00m
DLS083_Plate1_Green_B3_1_04d00h00m
DLS083_Plate1_Green_B3_1_05d00h00m
DLS083_Plate1_Green_B3_1_06d00h00m


In [15]:
# Get the tiffs to process
files = file_list[120:140]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_120to140.csv')

DLS083_Plate1_Green_B3_1_07d00h00m
DLS083_Plate1_Green_B3_1_08d00h00m
DLS083_Plate1_Green_B3_1_09d00h00m
DLS083_Plate1_Green_B3_1_10d00h00m
DLS083_Plate1_Green_B3_1_11d00h00m
DLS083_Plate1_Green_B3_1_12d00h00m
DLS083_Plate1_Green_B3_1_13d00h00m
DLS083_Plate1_Green_B3_1_14d00h00m
DLS083_Plate1_Green_B3_1_15d00h00m
DLS083_Plate1_Green_B3_1_16d00h00m
DLS083_Plate1_Green_B3_1_17d00h00m
DLS083_Plate1_Green_B3_1_18d00h00m
DLS083_Plate1_Green_B3_1_19d00h00m
DLS083_Plate1_Green_B3_1_20d00h00m
DLS083_Plate1_Green_B3_1_21d00h00m
DLS083_Plate1_Green_B3_1_22d00h00m
DLS083_Plate1_Green_B3_1_23d00h00m
DLS083_Plate1_Green_B3_1_24d00h00m
DLS083_Plate1_Green_B3_1_25d00h00m
DLS083_Plate1_Green_B3_1_26d00h00m


In [16]:
# Get the tiffs to process
files = file_list[140:160]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_140to160.csv')

DLS083_Plate1_Green_B3_1_27d00h00m
DLS083_Plate1_Green_B3_1_28d00h00m
DLS083_Plate1_Green_B3_1_29d00h00m
DLS083_Plate1_Green_B3_1_30d00h00m
DLS083_Plate2_GFP_A1_1_00d00h00m
DLS083_Plate2_GFP_A1_1_01d00h00m
DLS083_Plate2_GFP_A1_1_02d00h00m
DLS083_Plate2_GFP_A1_1_03d00h00m
DLS083_Plate2_GFP_A1_1_04d00h00m
DLS083_Plate2_GFP_A1_1_05d00h00m
DLS083_Plate2_GFP_A1_1_06d00h00m
DLS083_Plate2_GFP_A1_1_07d00h00m
DLS083_Plate2_GFP_A1_1_08d00h00m
DLS083_Plate2_GFP_A1_1_09d00h00m
DLS083_Plate2_GFP_A1_1_10d00h00m
DLS083_Plate2_GFP_A1_1_11d00h00m
DLS083_Plate2_GFP_A1_1_12d00h00m
DLS083_Plate2_GFP_A1_1_13d00h00m
DLS083_Plate2_GFP_A1_1_14d00h00m
DLS083_Plate2_GFP_A1_1_15d00h00m


In [17]:
# Get the tiffs to process
files = file_list[160:180]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_160to180.csv')

DLS083_Plate2_GFP_A1_1_16d00h00m
DLS083_Plate2_GFP_A1_1_17d00h00m
DLS083_Plate2_GFP_A1_1_18d00h00m
DLS083_Plate2_GFP_A1_1_19d00h00m
DLS083_Plate2_GFP_A1_1_20d00h50m
DLS083_Plate2_GFP_A1_1_21d00h50m
DLS083_Plate2_GFP_A1_1_22d00h50m
DLS083_Plate2_GFP_A1_1_23d00h50m
DLS083_Plate2_GFP_A1_1_24d00h50m
DLS083_Plate2_GFP_A1_1_25d00h50m
DLS083_Plate2_GFP_A1_1_26d00h50m
DLS083_Plate2_GFP_A1_1_27d00h50m
DLS083_Plate2_GFP_A1_1_28d00h50m
DLS083_Plate2_GFP_A1_1_29d00h50m
DLS083_Plate2_GFP_A1_1_30d00h50m
DLS083_Plate2_GFP_A2_1_00d00h00m
DLS083_Plate2_GFP_A2_1_01d00h00m
DLS083_Plate2_GFP_A2_1_02d00h00m
DLS083_Plate2_GFP_A2_1_03d00h00m
DLS083_Plate2_GFP_A2_1_04d00h00m


In [18]:
# Get the tiffs to process
files = file_list[180:200]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_180to200.csv')

DLS083_Plate2_GFP_A2_1_05d00h00m
DLS083_Plate2_GFP_A2_1_06d00h00m
DLS083_Plate2_GFP_A2_1_07d00h00m
DLS083_Plate2_GFP_A2_1_08d00h00m
DLS083_Plate2_GFP_A2_1_09d00h00m
DLS083_Plate2_GFP_A2_1_10d00h00m
DLS083_Plate2_GFP_A2_1_11d00h00m
DLS083_Plate2_GFP_A2_1_12d00h00m
DLS083_Plate2_GFP_A2_1_13d00h00m
DLS083_Plate2_GFP_A2_1_14d00h00m
DLS083_Plate2_GFP_A2_1_15d00h00m
DLS083_Plate2_GFP_A2_1_16d00h00m
DLS083_Plate2_GFP_A2_1_17d00h00m
DLS083_Plate2_GFP_A2_1_18d00h00m
DLS083_Plate2_GFP_A2_1_19d00h00m
DLS083_Plate2_GFP_A2_1_20d00h50m
DLS083_Plate2_GFP_A2_1_21d00h50m
DLS083_Plate2_GFP_A2_1_22d00h50m
DLS083_Plate2_GFP_A2_1_23d00h50m
DLS083_Plate2_GFP_A2_1_24d00h50m


In [24]:
# Get the tiffs to process
files = file_list[200:220]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_200to220.csv')

DLS083_Plate2_GFP_A2_1_25d00h50m
DLS083_Plate2_GFP_A2_1_26d00h50m
DLS083_Plate2_GFP_A2_1_27d00h50m
DLS083_Plate2_GFP_A2_1_28d00h50m
DLS083_Plate2_GFP_A2_1_29d00h50m
DLS083_Plate2_GFP_A2_1_30d00h50m
DLS083_Plate2_GFP_A3_1_00d00h00m
DLS083_Plate2_GFP_A3_1_01d00h00m
DLS083_Plate2_GFP_A3_1_02d00h00m
DLS083_Plate2_GFP_A3_1_03d00h00m
DLS083_Plate2_GFP_A3_1_04d00h00m
DLS083_Plate2_GFP_A3_1_05d00h00m
DLS083_Plate2_GFP_A3_1_06d00h00m
DLS083_Plate2_GFP_A3_1_07d00h00m
DLS083_Plate2_GFP_A3_1_08d00h00m
DLS083_Plate2_GFP_A3_1_09d00h00m
DLS083_Plate2_GFP_A3_1_10d00h00m
DLS083_Plate2_GFP_A3_1_11d00h00m
DLS083_Plate2_GFP_A3_1_12d00h00m
DLS083_Plate2_GFP_A3_1_13d00h00m


In [25]:
# Get the tiffs to process
files = file_list[220:240]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_220to240.csv')

DLS083_Plate2_GFP_A3_1_14d00h00m
DLS083_Plate2_GFP_A3_1_15d00h00m
DLS083_Plate2_GFP_A3_1_16d00h00m
DLS083_Plate2_GFP_A3_1_17d00h00m
DLS083_Plate2_GFP_A3_1_18d00h00m
DLS083_Plate2_GFP_A3_1_19d00h00m
DLS083_Plate2_GFP_A3_1_20d00h50m
DLS083_Plate2_GFP_A3_1_21d00h50m
DLS083_Plate2_GFP_A3_1_22d00h50m
DLS083_Plate2_GFP_A3_1_23d00h50m
DLS083_Plate2_GFP_A3_1_24d00h50m
DLS083_Plate2_GFP_A3_1_25d00h50m
DLS083_Plate2_GFP_A3_1_26d00h50m
DLS083_Plate2_GFP_A3_1_27d00h50m
DLS083_Plate2_GFP_A3_1_28d00h50m
DLS083_Plate2_GFP_A3_1_29d00h50m
DLS083_Plate2_GFP_A3_1_30d00h50m
DLS083_Plate2_GFP_B1_1_00d00h00m
DLS083_Plate2_GFP_B1_1_01d00h00m
DLS083_Plate2_GFP_B1_1_02d00h00m


In [26]:
# Get the tiffs to process
files = file_list[240:260]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_240to260.csv')

DLS083_Plate2_GFP_B1_1_03d00h00m
DLS083_Plate2_GFP_B1_1_04d00h00m
DLS083_Plate2_GFP_B1_1_05d00h00m
DLS083_Plate2_GFP_B1_1_06d00h00m
DLS083_Plate2_GFP_B1_1_07d00h00m
DLS083_Plate2_GFP_B1_1_08d00h00m
DLS083_Plate2_GFP_B1_1_09d00h00m
DLS083_Plate2_GFP_B1_1_10d00h00m
DLS083_Plate2_GFP_B1_1_11d00h00m
DLS083_Plate2_GFP_B1_1_12d00h00m
DLS083_Plate2_GFP_B1_1_13d00h00m
DLS083_Plate2_GFP_B1_1_14d00h00m
DLS083_Plate2_GFP_B1_1_15d00h00m
DLS083_Plate2_GFP_B1_1_16d00h00m
DLS083_Plate2_GFP_B1_1_17d00h00m
DLS083_Plate2_GFP_B1_1_18d00h00m
DLS083_Plate2_GFP_B1_1_19d00h00m
DLS083_Plate2_GFP_B1_1_20d00h50m
DLS083_Plate2_GFP_B1_1_21d00h50m
DLS083_Plate2_GFP_B1_1_22d00h50m


In [27]:
# Get the tiffs to process
files = file_list[260:280]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_260to280.csv')

DLS083_Plate2_GFP_B1_1_23d00h50m
DLS083_Plate2_GFP_B1_1_24d00h50m
DLS083_Plate2_GFP_B1_1_25d00h50m
DLS083_Plate2_GFP_B1_1_26d00h50m
DLS083_Plate2_GFP_B1_1_27d00h50m
DLS083_Plate2_GFP_B1_1_28d00h50m
DLS083_Plate2_GFP_B1_1_29d00h50m
DLS083_Plate2_GFP_B1_1_30d00h50m
DLS083_Plate2_GFP_B2_1_00d00h00m
DLS083_Plate2_GFP_B2_1_01d00h00m
DLS083_Plate2_GFP_B2_1_02d00h00m
DLS083_Plate2_GFP_B2_1_03d00h00m
DLS083_Plate2_GFP_B2_1_04d00h00m
DLS083_Plate2_GFP_B2_1_05d00h00m
DLS083_Plate2_GFP_B2_1_06d00h00m
DLS083_Plate2_GFP_B2_1_07d00h00m
DLS083_Plate2_GFP_B2_1_08d00h00m
DLS083_Plate2_GFP_B2_1_09d00h00m
DLS083_Plate2_GFP_B2_1_10d00h00m
DLS083_Plate2_GFP_B2_1_11d00h00m


In [28]:
# Get the tiffs to process
files = file_list[280:300]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_280to300.csv')

DLS083_Plate2_GFP_B2_1_12d00h00m
DLS083_Plate2_GFP_B2_1_13d00h00m
DLS083_Plate2_GFP_B2_1_14d00h00m
DLS083_Plate2_GFP_B2_1_15d00h00m
DLS083_Plate2_GFP_B2_1_16d00h00m
DLS083_Plate2_GFP_B2_1_17d00h00m
DLS083_Plate2_GFP_B2_1_18d00h00m
DLS083_Plate2_GFP_B2_1_19d00h00m
DLS083_Plate2_GFP_B2_1_20d00h50m
DLS083_Plate2_GFP_B2_1_21d00h50m
DLS083_Plate2_GFP_B2_1_22d00h50m
DLS083_Plate2_GFP_B2_1_23d00h50m
DLS083_Plate2_GFP_B2_1_24d00h50m
DLS083_Plate2_GFP_B2_1_25d00h50m
DLS083_Plate2_GFP_B2_1_26d00h50m
DLS083_Plate2_GFP_B2_1_27d00h50m
DLS083_Plate2_GFP_B2_1_28d00h50m
DLS083_Plate2_GFP_B2_1_29d00h50m
DLS083_Plate2_GFP_B2_1_30d00h50m
DLS083_Plate2_GFP_B3_1_00d00h00m


In [29]:
# Get the tiffs to process
files = file_list[300:320]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_300to320.csv')

DLS083_Plate2_GFP_B3_1_01d00h00m
DLS083_Plate2_GFP_B3_1_02d00h00m
DLS083_Plate2_GFP_B3_1_03d00h00m
DLS083_Plate2_GFP_B3_1_04d00h00m
DLS083_Plate2_GFP_B3_1_05d00h00m
DLS083_Plate2_GFP_B3_1_06d00h00m
DLS083_Plate2_GFP_B3_1_07d00h00m
DLS083_Plate2_GFP_B3_1_08d00h00m
DLS083_Plate2_GFP_B3_1_09d00h00m
DLS083_Plate2_GFP_B3_1_10d00h00m
DLS083_Plate2_GFP_B3_1_11d00h00m
DLS083_Plate2_GFP_B3_1_12d00h00m
DLS083_Plate2_GFP_B3_1_13d00h00m
DLS083_Plate2_GFP_B3_1_14d00h00m
DLS083_Plate2_GFP_B3_1_15d00h00m
DLS083_Plate2_GFP_B3_1_16d00h00m
DLS083_Plate2_GFP_B3_1_17d00h00m
DLS083_Plate2_GFP_B3_1_18d00h00m
DLS083_Plate2_GFP_B3_1_19d00h00m
DLS083_Plate2_GFP_B3_1_20d00h50m


In [30]:
# Get the tiffs to process
files = file_list[320:340]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_320to340.csv')

DLS083_Plate2_GFP_B3_1_21d00h50m
DLS083_Plate2_GFP_B3_1_22d00h50m
DLS083_Plate2_GFP_B3_1_23d00h50m
DLS083_Plate2_GFP_B3_1_24d00h50m
DLS083_Plate2_GFP_B3_1_25d00h50m
DLS083_Plate2_GFP_B3_1_26d00h50m
DLS083_Plate2_GFP_B3_1_27d00h50m
DLS083_Plate2_GFP_B3_1_28d00h50m
DLS083_Plate2_GFP_B3_1_29d00h50m
DLS083_Plate2_GFP_B3_1_30d00h50m
DLS083_Plate1_Green_A1_1_00d00h00m
DLS083_Plate1_Green_A1_1_01d00h00m
DLS083_Plate1_Green_A1_1_02d00h00m
DLS083_Plate1_Green_A1_1_03d00h00m
DLS083_Plate1_Green_A1_1_04d00h00m
DLS083_Plate1_Green_A1_1_05d00h00m
DLS083_Plate1_Green_A1_1_06d00h00m
DLS083_Plate1_Green_A1_1_07d00h00m
DLS083_Plate1_Green_A1_1_08d00h00m
DLS083_Plate1_Green_A1_1_09d00h00m


In [31]:
# Get the tiffs to process
files = file_list[340:360]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_340to360.csv')

DLS083_Plate1_Green_A1_1_10d00h00m
DLS083_Plate1_Green_A1_1_11d00h00m
DLS083_Plate1_Green_A1_1_12d00h00m
DLS083_Plate1_Green_A1_1_13d00h00m
DLS083_Plate1_Green_A1_1_14d00h00m
DLS083_Plate1_Green_A1_1_15d00h00m
DLS083_Plate1_Green_A1_1_16d00h00m
DLS083_Plate1_Green_A1_1_17d00h00m
DLS083_Plate1_Green_A1_1_18d00h00m
DLS083_Plate1_Green_A1_1_19d00h00m
DLS083_Plate1_Green_A1_1_20d00h00m
DLS083_Plate1_Green_A1_1_21d00h00m
DLS083_Plate1_Green_A1_1_22d00h00m
DLS083_Plate1_Green_A1_1_23d00h00m
DLS083_Plate1_Green_A1_1_24d00h00m
DLS083_Plate1_Green_A1_1_25d00h00m
DLS083_Plate1_Green_A1_1_26d00h00m
DLS083_Plate1_Green_A1_1_27d00h00m
DLS083_Plate1_Green_A1_1_28d00h00m
DLS083_Plate1_Green_A1_1_29d00h00m


In [32]:
# Get the tiffs to process
files = file_list[360:]

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    ax = fig.add_subplot(5, 2, 1)
    imgplot = plt.imshow(image)
    imgplot.set_clim(0, 100)
    ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    imgplot = plt.imshow(cut_im)
    imgplot.set_clim(0, 100)
    ax.set_title('Cut image')


    # Gaussian filter and then plot
    ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    imgplot = plt.imshow(image_gauss)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    imgplot = plt.imshow(image_gauss_median)
    imgplot.set_clim(0.0, .0025)
    ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.02
    imgplot = plt.imshow(binary_temp)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    #too_large = cell_sizes > max_size # Removed beause this messes with conditions that lose contact inhibition
    #too_large_mask = too_large[ccs]
    #binary[too_large_mask] = 0
    imgplot = plt.imshow(binary)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled')

    # remove border of false cells
    ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(450,450),(h-450,w-450)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    imgplot = plt.imshow(binary_crop)
    imgplot.set_clim(0, 1)
    ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

    plt.close()

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_360toend.csv')

DLS083_Plate1_Green_A1_1_30d00h00m
DLS083_Plate1_Green_A2_1_00d00h00m
DLS083_Plate1_Green_A2_1_01d00h00m
DLS083_Plate1_Green_A2_1_02d00h00m
DLS083_Plate1_Green_A2_1_03d00h00m
DLS083_Plate1_Green_A2_1_04d00h00m
DLS083_Plate1_Green_A2_1_05d00h00m
DLS083_Plate1_Green_A2_1_06d00h00m
DLS083_Plate1_Green_A2_1_07d00h00m
DLS083_Plate1_Green_A2_1_08d00h00m
DLS083_Plate1_Green_A2_1_09d00h00m
DLS083_Plate1_Green_A2_1_10d00h00m
